# HDS5210-2024 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 24th.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [12]:
def priest(gender, age, breath_rate, spO2, heart_rate, sys_BP, temp, alert_level, oxygen_type, activity_level):
    """
    (string, int, float, float, float, float, float, string, string, string) -> (float)

    This function calculates the risk of a severe outcome (like hospitalization or death) within 30 days using
    the PRIEST COVID-19 Clinical Severity Score. It takes several health and demographic factors as input
    and gives a risk score based on them. The score is then mapped to a probability of a severe outcome.

    Input Arguments:
        gender (str): 'male' or 'female'.
        age (int): Age in years.
        breath_rate (float): Respiratory rate in breaths per minute.
        spO2 (float): Oxygen saturation, scale 0 to 1.
        heart_rate (float): Heart rate in beats per minute.
        sys_BP (float): Systolic blood pressure in mmHg.
        temp (float): Body temperature in Celsius.
        alert_level (str): 'alert' or 'confused or not alert'.
        oxygen_type (str): 'air' or 'supplemental oxygen'.
        activity_level (str): Physical activity level description.

    Output of the function:
        float: Estimated probability of severe outcome within 30 days.

    Examples:
        >>> priest('MALE', 90, 66, 0.90, 88, 89, 40.0, 'confused or not alert', 'supplemental oxygen', 'limited self-care')
        0.59

        >>> priest('male', 20, 79, 0.66, 40, 150, 39.5, 'alert', 'AIR', 'unrestricted normal activity')
        0.38

        >>> priest('female', 77, 56, 0.66, 86, 92, 20.0, 'alert', 'supplemental oxygen', 'limited self-care')
        0.59

        >>> priest('male', 30, 79, 0.96, 20, 110, 33.5, 'ALERT', 'AIR', 'unrestricted normal activity')
        0.34

    """

    # Initialize PRIEST score
    score = 1 if gender.lower() == 'male' else 0

    # Age-related scoring
    score += 2 if 50 <= age <= 65 else 3 if 66 <= age <= 80 else 4 if age > 80 else 0

    # Respiratory rate scoring
    score += 3 if breath_rate < 9 or breath_rate > 24 else 2 if 21 <= breath_rate <= 24 else 1 if 9 <= breath_rate <= 11 else 0

    # Oxygen saturation scoring
    score += 3 if spO2 < 0.92 else 2 if 0.92 <= spO2 <= 0.93 else 1 if 0.94 <= spO2 <= 0.95 else 0

    # Heart rate scoring
    score += 3 if heart_rate < 41 or heart_rate > 130 else 2 if 111 <= heart_rate <= 130 else 1 if 41 <= heart_rate <= 50 or 91 <= heart_rate <= 110 else 0

    # Blood pressure scoring
    score += 3 if sys_BP < 91 or sys_BP > 219 else 2 if 91 <= sys_BP <= 100 else 1 if 101 <= sys_BP <= 110 else 0

    # Temperature scoring
    score += 3 if temp < 35.1 else 2 if temp > 39.0 else 1 if 35.1 <= temp <= 36.0 or 38.1 <= temp <= 39.0 else 0

    # Alertness scoring
    score += 3 if alert_level.lower() != 'alert' else 0

    # Oxygen intake scoring
    score += 2 if oxygen_type.lower() == 'supplemental oxygen' else 0

    # Mobility scoring
    mobility_scores = {
        'unrestricted normal activity': 0,
        'limited strenuous activity, can do light activity': 1,
        'limited activity, can self-care': 2,
        'limited self-care': 3,
        'bed/chair bound, no self-care': 4
    }
    score += mobility_scores.get(activity_level.lower(), 0)

    # Probability mapping
    probabilities = {1: 0.01, 2: 0.02, 3: 0.03, 4: 0.03, 5: 0.09, 6: 0.15, 7: 0.18, 8: 0.22, 9: 0.26, 10: 0.29, 11: 0.34,
                     12: 0.38, 13: 0.46, 14: 0.47, 15: 0.49, 16: 0.55, 26: 0.99}
    return probabilities.get(score, 0.59 if 17 <= score <= 25 else "Score not found")


In [13]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest('MALE', 90, 66, 0.90, 88, 89, 40.0, 'confused or not alert', 'supplemental oxygen', 'limited self-care')
Expecting:
    0.59
ok
Trying:
    priest('male', 20, 79, 0.66, 40, 150, 39.5, 'alert', 'AIR', 'unrestricted normal activity')
Expecting:
    0.38
ok
Trying:
    priest('female', 77, 56, 0.66, 86, 92, 20.0, 'alert', 'supplemental oxygen', 'limited self-care')
Expecting:
    0.59
ok
Trying:
    priest('male', 30, 79, 0.96, 20, 110, 33.5, 'ALERT', 'AIR', 'unrestricted normal activity')
Expecting:
    0.34
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [14]:
import requests

def find_hospital(age, sex, risk_pct):
    """
    (int, string, float) -> (String)
    This function finds the appropriate hospital for a patient based on their age, sex, and risk percentage.
    It sends a request to a REST web service, which returns the name of the hospital where the patient should go.

    Input Arguments:
        age (int): The age of the patient.
        sex (str): The sex of the patient ('male' or 'female').
        risk_pct (float): The risk percentage of the patient.

    Output of the function:
        str: The name of the hospital or None if the request fails.

    Test cases:
    >>> find_hospital(20, 'female', 0.92)
    'Emory Dunwoody Medical Center'

    >>> find_hospital(80, 'female', 0.34)
    'Wesley Woods Geriatric Hospital'

    >>> find_hospital(50, 'female', 0.2)
    'Select Specialty Hospital - Northeast Atlanta'
    """
    base_url = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/"
    params = {
        'age': age,
        'sex': sex,
        'risk_pct': risk_pct
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raises a HTTPError for bad responses
        data = response.json()
        return data.get('hospital')
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

In [15]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(20, 'female', 0.92)  
Expecting:
    'Emory Dunwoody Medical Center'
ok
Trying:
    find_hospital(80, 'female', 0.34) 
Expecting:
    'Wesley Woods Geriatric Hospital'
ok
Trying:
    find_hospital(50, 'female', 0.2)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [16]:
import requests

def get_address(hospital_name):
    """
    (String) -> (String)

    The function looks up the address of a hospital based on its name.
    It retrieves hospital information from an online JSON file.
    If the hospital is found, it returns the address.
    If the hospital is not found or there’s an error, it returns None.
    The function handles network issues and returns a clear result for each hospital query.

    Input Arguments::
        hospital_name (str): The name of the hospital for which the address is required.

    Output of the function::
        str: The address of the hospital or None if the hospital is not found.

    Test cases:

    >>> get_address("CANDLER COUNTY HOSPITAL")
    '400 CEDAR ROAD'

    >>> get_address("CLEARVIEW REGIONAL MEDICAL CENTER")
    '330 ALCOVY STREET'

    >>> get_address("TURNING POINT HOSPITAL")
    '3015 VETERANS PKWY SOUTH'
    """

    url = "https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises HTTPError for bad requests
        hospitals = response.json()
        hospital_info = hospitals.get(hospital_name.upper(), None)
        if hospital_info:
            return hospital_info['ADDRESS']
        else:
            return None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

In [17]:
import doctest
doctest.run_docstring_examples(get_address, globals(),verbose=True)

Finding tests in NoName
Trying:
    get_address("CANDLER COUNTY HOSPITAL")
Expecting:
    '400 CEDAR ROAD'
ok
Trying:
    get_address("CLEARVIEW REGIONAL MEDICAL CENTER") 
Expecting:
    '330 ALCOVY STREET'
ok
Trying:
    get_address("TURNING POINT HOSPITAL") 
Expecting:
    '3015 VETERANS PKWY SOUTH'
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [18]:
import requests

def process_people(file_url):
    """
    This function processes a pipe-delimited file containing person data to calculate the risk, find appropriate hospitals,
    and addresses for each person in the file.

    Parameters:
        file_url (str): The URL to download the person data file.

    Returns:
        list: A list of dictionaries, each containing patient details and their assigned hospital and address.
    """
    # Retrieve the people.psv file from the provided URL
    try:
        response = requests.get(file_url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Failed to download file: {e}")
        return []

    # Split lines and process data
    lines = response.text.strip().split('\n')
    headers = lines[0].split('|')  # This assumes the first line is the header
    patients_data = []

    # Process each line after the header
    for line in lines[1:]:
        fields = line.split('|')
        patient_dict = {headers[i]: fields[i] for i in range(len(headers))}
        patient_id = patient_dict['patient']
        sex = patient_dict['sex']
        age = int(patient_dict['age'])
        breath = int(patient_dict['breath'])
        o2sat = float(patient_dict['o2 sat'])
        heart = int(patient_dict['heart'])
        systolic = int(patient_dict['systolic bp'])
        temp = float(patient_dict['temp'])
        alertness = patient_dict['alertness']
        inspired = patient_dict['inspired']
        status = patient_dict['status']

        # Calculate the risk using a fictitious PRIEST function (assuming it's defined)
        risk = priest(sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status)

        # Find the hospital and address using the previously defined functions
        hospital = find_hospital(age, sex, risk)
        address = get_address(hospital)

        # Collect all information into the list
        patients_data.append({
            patient_id: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]
        })

    return patients_data


## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [22]:
import requests
import json

def download_json_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as error:
        print(f"Error downloading JSON data: {error}")
        return None

def check_hospital_data(actual_data_list, expected_data):
    """Compares hospital names and addresses between actual and expected results, providing detailed feedback for each entry."""
    results = {}

    # Convert list of dictionaries (actual_data_list) to a single dictionary (actual_data_dict)
    actual_data_dict = {key: value for item in actual_data_list for key, value in item.items()}

    for patient_id, actual_values in actual_data_dict.items():
        expected_values = expected_data.get(patient_id)
        result = {"Match": False, "Details": {}}

        if not expected_values:
            result["Details"] = "Patient ID not found in expected results."
        else:
            actual_hospital, actual_address = actual_values[-2], actual_values[-1]
            expected_hospital, expected_address = expected_values[-2], expected_values[-1]

            if actual_hospital == expected_hospital and actual_address == expected_address:
                result["Match"] = True
            else:
                result["Details"] = {
                    'Actual Hospital': actual_hospital,
                    'Expected Hospital': expected_hospital,
                    'Actual Address': actual_address,
                    'Expected Address': expected_address
                }

        results[patient_id] = result

    return results

# URL to download the expected results
expected_results_url = "https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5"
expected_results = download_json_from_url(expected_results_url)

file_url = "https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh"
actual_results = process_people(file_url)

# Check each patient's data and print the output
patient_results = check_hospital_data(actual_results, expected_results)
for patient_id, info in patient_results.items():
    if info["Match"]:
        print(f"Patient ID {patient_id}: Data matches.")
    else:
        print(f"Patient ID {patient_id}: Discrepancy found.")
        print(json.dumps(info["Details"], indent=4))

Patient ID E9559: Data matches.
Patient ID E9385: Data matches.
Patient ID E3067: Data matches.
Patient ID E9422: Data matches.
Patient ID E8661: Data matches.
Patient ID E6235: Data matches.
Patient ID E4451: Data matches.
Patient ID E8433: Data matches.
Patient ID E7593: Data matches.
Patient ID E3296: Data matches.
Patient ID E4157: Data matches.
Patient ID E7702: Data matches.
Patient ID E8158: Data matches.
Patient ID E4795: Data matches.
Patient ID E8800: Data matches.
Patient ID E4855: Data matches.
Patient ID E9691: Data matches.
Patient ID E4535: Data matches.
Patient ID E2360: Data matches.
Patient ID E3447: Data matches.
Patient ID E8208: Data matches.
Patient ID E4428: Data matches.
Patient ID E2766: Data matches.
Patient ID E3888: Data matches.
Patient ID E7110: Data matches.
Patient ID E2668: Data matches.
Patient ID E8482: Data matches.
Patient ID E9619: Data matches.
Patient ID E1700: Data matches.
Patient ID E8585: Data matches.
Patient ID E8633: Data matches.
Patient 

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---